In [1]:
.include header

In [2]:
%%file dynamic-vector.h

#pragma once

#include <vector>
#include <array>

namespace eliastocco { // you can change namespace    
       
    namespace container {
    
        namespace dynamic {
        
            typedef /*long unsigned*/ int t_dim;
            typedef /*long*/ long unsigned int ut_dim;
        
            constexpr ut_dim decrease ( ut_dim a ) { return (a > 1 ) ? ( a - 1 ) : 0 ; }
            
            template < class type , template <class...> class STL_container >
                class my_dynamic_vector ;
                
            template < ut_dim Dimension , class type , 
                template <class...> class STL_container > 
                    class dynamic_vector ;
        
        
        }
        
    }
    
}

#include "header/my_dynamic_vector.h"
#include "header/dynamic_vector.h"
#include "header/dynamic_vector_specialization.h"

Overwriting dynamic-vector.h


In [3]:
%%file header/my_dynamic_vector.h

#pragma once

namespace eliastocco {       
    namespace container {    
        namespace dynamic {
        
            // do not use this class
            
            template < class type , template <class...> class STL_container >
                class my_dynamic_vector : public STL_container < type > {
                    public :
                        my_dynamic_vector ( std :: array < t_dim , 1 > Lenght = { 0 } ) 
                            : STL_container < type > ( Lenght [ 0 ] ) {};
                };
            
        }        
    }
}

Overwriting header/my_dynamic_vector.h


In [4]:
%%file header/dynamic_vector.h

#pragma once

#define thisclass dynamic_vector<Dimension,type,STL_container>                        
#define motherclass my_dynamic_vector<dynamic_vector<decrease(Dimension),\
type,STL_container>,STL_container>                        
#define childclass dynamic_vector<decrease(Dimension),type,STL_container>

namespace eliastocco {       
    namespace container {    
        namespace dynamic {
            
            //inherit an "array" of dimension Dimension -1 
            template < ut_dim Dimension , class type , 
                template <class...> class STL_container = std :: vector > 
                    class dynamic_vector 
                        : public my_dynamic_vector 
                            < dynamic_vector < decrease(Dimension) , 
                            type , STL_container > , STL_container > {                        
                
                
                    static_assert ( Dimension > 0 , "Dimension should be > 0" ) ;

                    public :
                        typedef std :: array < t_dim , Dimension > Index_t;

                    public :
                        dynamic_vector ( Index_t Lenght = { 0 } ) {
                            std :: array < t_dim , decrease(Dimension) > Other = { 0 } ;
                                for ( auto i = 0 ; i != Other . size () ; i ++ ) {
                                    Other [ i ] = Lenght [ i + 1 ] ;
                                }
                            my_dynamic_vector<
                                dynamic_vector<decrease(Dimension),type,STL_container>,STL_container>
                                    :: resize (Lenght[0],
                                        dynamic_vector<decrease(Dimension),type,STL_container>(Other));

                        };

                    //operators
                    public : childclass & operator[](const t_dim index );
                    public : childclass   operator[](const t_dim index ) const ;
                    public : template < ut_dim Dim > 
                                auto & operator[](const std :: array<t_dim,Dim> index );
                    public : template < ut_dim Dim > 
                                auto   operator[](const std :: array<t_dim,Dim> index ) const ;

                    //functions
                    public : thisclass transpose (const t_dim left=0 , const t_dim right=1) const ;
                    public : Index_t begin() const ;
                    public : Index_t next(const Index_t now ) const ;
                    public : Index_t last() const ;
                    public : Index_t end() const ;
                        
            } ;
            
        }        
    }
}
#define Template template<ut_dim Dimension,class type,template <class...> class STL_container >
#define Class dynamic_vector <Dimension,type,STL_container>  

#include "dynamic_vector_operators.h"
#include "dynamic_vector_functions.h"

#undef Template
#undef Class

#undef motherclass 
#undef childclass
#undef thisclass

Overwriting header/dynamic_vector.h


In [5]:
%%file header/dynamic_vector_operators.h

#pragma once

namespace eliastocco {       
    namespace container {    
        namespace dynamic {
        
            //#define Class template<ut_dim Dimension,class type,\
            //    template <class...> class STL_container >\
            //        dynamic_vector <Dimension,type,STL_container>
                 
                Template
                    childclass & Class :: operator[](const t_dim index ){
                        //std :: cout << "childclass & operator[](const t_dim index ) "
                        //<< index << std::endl;
                        //return motherclass::operator[](index);
                        return motherclass::at(index);
                    }

                Template
                    childclass Class :: operator[](const t_dim index ) const {
                        //std :: cout << "childclass operator[](const t_dim index ) 
                        //<<index<<std::endl;
                        //return motherclass::operator[](index);
                        return motherclass::at(index);
                    }
    
        
                Template
                    template < ut_dim Dim >
                        auto & Class :: operator[](const std :: array<t_dim,Dim> index ) {
                            //std::cout<< "auto & operator[](const std :: array<t_dim,Dim> index )Dim=" 
                            //<< Dim << std :: endl;

                            if constexpr (Dim>1){

                                std :: array < t_dim , decrease(Dim) > new_index = { 0 } ;
                                for ( auto i = 0 ; i < Dim-1 ; i ++ ) {
                                    new_index [ i ] = index [ i + 1 ] ;
                                }

                                //auto a = motherclass::operator[](index[0]);

                                //return motherclass::operator[](index[0])[new_index];
                                return (motherclass::at(index[0]))[new_index];

                            }else {
                                //std :: cout << "ciao2" << index[0] << std :: endl;
                                return motherclass::at(index[0]);
                            }

                        }

                Template
                    template < ut_dim Dim >
                        auto Class :: operator[](const std :: array<t_dim,Dim> index ) const {
                            //std :: cout << "auto operator[](const std :: array<t_dim,Dim> index ) 
                            //const, Dim=" << Dim << std :: endl;

                            if constexpr (Dim>1){

                                std :: array < t_dim , decrease(Dim) > new_index = { 0 } ;
                                for ( auto i = 0 ; i < Dim-1 ; i ++ ) {
                                    new_index [ i ] = index [ i + 1 ] ;
                                }

                                //std :: cout << "ciao" << index[0] << std :: endl;

                                //auto a = motherclass::operator[](index[0]);

                                //return motherclass::operator[](index[0])[new_index];
                                return (motherclass::at(index[0]))[new_index];

                            }else {
                                //std :: cout << "ciao2" << index[0] << std :: endl;
                                return motherclass::at(index[0]);
                            }

                        }
                            
            //#undef Class
        
        }
    }
}

Overwriting header/dynamic_vector_operators.h


In [6]:
%%file header/dynamic_vector_functions.h

#pragma once

namespace eliastocco {       
    namespace container {    
        namespace dynamic {
        
            //#define Template template<ut_dim Dimension,class type,\
            //    template <class...> class STL_container >\
            //        dynamic_vector <Dimension,type,STL_container>
            
            #define Index_t typename Class::Index_t
             
            Template
                thisclass Class :: transpose (const t_dim left , const t_dim right ) const {

                    if(left==right){return *this;}
                    //std :: cout << "transpose" << std :: endl ;
                    auto e = this->end();
                    auto _e = this->last();
                    for(auto i=0;i<Dimension;++i){
                        _e[i]++;
                    }
                    std :: array < t_dim , Dimension > Lenght = _e;                            
                    Lenght[left] = _e[right];
                    Lenght[right]= _e[left];                            

                    thisclass out (Lenght);           

                    //std :: cout << "out" << std :: endl ;
                    t_dim appo =0;

                    auto b = this->begin();                            
                    auto j = b; //solo per avere il tipo giusto                            
                    //int k = 0 ;                            
                    //std :: cout << "cycle" << std :: endl ;
                    for(auto i=b;i!=e;i=this->next(i)){                            
                        //std :: cout << k ++  << std :: endl ;                                
                        //for(auto kk=0;kk<Dimension;++kk){
                        //    std::cout << i[kk] << ",";
                        //}
                        //std :: cout << std :: endl;                            
                        j=i;
                        j[left]=i[right];
                        j[right]=i[left];
                        //std::swap(j[left],i[right]);                                
                        out[j]=(*this)[i];

                    }

                    return out;

                }
            
            Template
                Index_t Class :: begin() const {                            
                    Index_t index; 
                    index.fill(0);
                    return index;
                }
                        
            Template
                Index_t Class :: next(const Index_t now ) const {

                    auto size=this->size();
                    if(now[0]>=size){                                
                        return this->end();//now
                    }

                    std :: array < t_dim , Dimension -1 > sub_index;
                    for(auto i=0;i<Dimension-1;++i){
                        sub_index[i]=now[i+1];
                    }

                    auto A = (*this)[now[0]];
                    auto a = A.next(sub_index);
                    //auto a = motherclass::operator[](now[0]).next(sub_index);
                    if (a==sub_index){                                
                        if(now[0]==size-1){
                            return this->end();//now
                        }else{
                            Index_t index;
                            index.fill(0);
                            index[0]=now[0]+1;
                            return index;
                        }                                
                    } else {
                        Index_t index;
                        std::copy(a.begin(),a.end(),std::next(index.begin()));
                        index[0]=now[0];
                        return index;
                    }
                }
                        
                Template
                    Index_t Class :: last() const {                        
                        Index_t now ;
                        Index_t last ;
                        Index_t index =this->begin(); 
                        do{
                            last=now;
                            now=index;
                            index = this->next(now);  
                        }while(index!=now);  
                        return last;
                    }
                    
                Template
                    Index_t Class :: end() const {                        
                        Index_t index;
                        index.fill(-1);                 
                        return index;
                    }   
              
              #undef Index_t
        }
    }
}

Overwriting header/dynamic_vector_functions.h


In [7]:
%%file header/dynamic_vector_specialization.h

#pragma once

namespace eliastocco {       
    namespace container {    
        namespace dynamic {
        
            // class specialization 
            template < class type , template <class...> class STL_container > 
                class dynamic_vector < 1 , type , STL_container > 
                    : public  my_dynamic_vector < type , STL_container > {

                    #define motherclass my_dynamic_vector<type,STL_container>
                    
                        public :
                            typedef std :: array < t_dim ,1> Index_t;

                        public :             
                            dynamic_vector ( std :: array < t_dim , 1 > Lenght = { 0 } ) 
                                : my_dynamic_vector < type , STL_container > ( Lenght ) {}

                        public :
                            Index_t begin() const {
                                Index_t index ;
                                index[0]=0;
                                return index;
                            }

                        public :
                            Index_t last() const {
                                Index_t index ;
                                index[0]=this->size();
                                return index;
                            }

                        public :
                            Index_t end() const {
                                Index_t index ;
                                index[0]=-1;
                                return index;
                            }

                        public :
                            Index_t next(const Index_t now ) const {

                                auto size = this->size();
                                Index_t index ;
                                if (now[0]>=size){ //out of range
                                    index[0]=-1;
                                }else if(now[0]==size-1){ // the last one
                                    index[0]=size-1;
                                }else{
                                    index[0]=now[0]+1; //return the next
                                }

                                return index;

                            }                           

                        public :
                            type & operator[](const t_dim index ){
                                //std :: cout << "type & operator[](const t_dim index ) " << index 
                                //<< std :: endl;   
                                //return motherclass::operator[](index);
                                return motherclass::at(index);
                            }

                        public :
                            type operator[](const t_dim index ) const {
                                //std :: cout << "type operator[](const t_dim index ) const " << index 
                                //<< std :: endl;   
                                //return motherclass::operator[](index);
                                return motherclass::at(index);
                            }

                        public :
                            type & operator[](const Index_t index ){
                                //std :: cout<<"type & operator[](const std :: array < t_dim,1> index ) " 
                                //<< index[0] << std :: endl;   
                                //return motherclass::operator[](index[0]);
                                return motherclass::at(index[0]);
                            }

                        public :
                            type operator[](const Index_t index ) const {
                                //std :: cout << "type operator[](const std :: array < t_dim,1> index ) 
                                //const " << index[0] <<std :: endl;  
                                //return motherclass::operator[](index[0]);
                                return motherclass::at(index[0]);
                            }
                            
                            
                        #undef motherclass  

            };  
        
        }
    }
}

Overwriting header/dynamic_vector_specialization.h


In [8]:
#include <iostream>
#include "dynamic-vector.h"
using namespace eliastocco :: container :: dynamic ;

In [9]:
std :: array<double,5> p = {2}

In [10]:
dynamic_vector < 2 , double > matrix = {{ 2 , 3 }};

In [11]:
dynamic_vector < 1 , double > matrix1 = {{ 2 }};

In [12]:
dynamic_vector < 1 , std::array<double,1> > prova = {{ 2 }};

In [13]:
std::array<long unsigned int,1> i = {0}

In [14]:
std::array<double,1> a = {0}

In [15]:
prova[0]=prova[0]

@0x7fffd5e9ed20

In [16]:
matrix1[0]=2;
matrix1[1]=3;

In [17]:
dynamic_vector < 2 , double > matrix2 = {{ 3 , 3 }};

In [18]:
std::array<long int,2> i2 = {0,0}

In [19]:
i2[0]=1;
i2[1]=0;

In [20]:
matrix2[0][0]

0.0000000

In [21]:
typeid(matrix2[1][2]).name()

"d"

In [22]:
auto b = matrix2.begin()

In [23]:
typeid(b).name()

"St5arrayIiLm2EE"

In [24]:
typeid(i2).name()

"St5arrayIlLm2EE"

In [25]:
matrix2[b]

0.0000000

In [26]:
matrix2.next(b)

@0x7fffd7be2220

In [27]:
auto l = matrix2.last()

In [28]:
l[1]

2

In [29]:
auto e = matrix2.end()

In [30]:
e[1]

-1

In [31]:
int k = 0 ;

In [32]:
for(auto i=b;i!=e;i=matrix2.next(i)){
    std :: cout << i[0] << "," << i[1] << " : ";
    //matrix2[i]=k++;
    std :: cout << matrix2[i] << " , " << std :: endl;
    //std :: cout << t[i] << std :: endl;
}

0,0 : 0 , 
0,1 : 0 , 
0,2 : 0 , 
1,0 : 0 , 
1,1 : 0 , 
1,2 : 0 , 
2,0 : 0 , 
2,1 : 0 , 
2,2 : 0 , 


In [33]:
auto t = matrix2.transpose()

In [13]:
matrix2[i2]=4;
matrix2[i2]

4.0000000

In [14]:
matrix2[1]=matrix1;
matrix2[1]

{ 2.0000000, 0.0000000 }

In [15]:
matrix2[i]=matrix1

{ 2.0000000, 0.0000000 }

In [16]:
matrix2[0][1]=3.0;
matrix2[0][1]

3.0000000

In [17]:
std::array<long unsigned int,3> init = { 2 , 3 , 4 }

In [18]:
dynamic_vector < 3 , double > matrix3 (init);//= {{ 2 , 3 , 4 }};

In [19]:
std::array<long unsigned int,3> i3 = {0,0,0}

In [20]:
matrix3[0][2][0]

0.0000000

In [21]:
matrix3[i2]=matrix1

{ 2.0000000, 0.0000000 }

In [22]:
matrix3[i3]

2.0000000

In [23]:
dynamic_vector < 2 , unsigned int > C ( { 6 , 6 } );

In [24]:
int N = 5

In [30]:


                        //std :: cout << "coefficienti binomiali" << std :: endl ; 
                        C[0][0] = 1;
                        for ( int n = 1; n <= N; ++n) {
                            C[n][0] = C[n][n] = 1;
                            for ( int k = 1; k < n; ++k){
                                C[n][k] = C[n - 1][k - 1] + C[n - 1][k];
                            }
                        }  

In [6]:
dynamic_vector < 2 , double , std :: vector > matrix2 = {{ 2 , 3 }}

In [ ]:
dynamic_vector < 2 , double , std :: deque > matrix3 = {{ 2 , 3 }};
std :: cout << matrix3.size() << std :: endl ;
std :: cout << matrix3[0].size() << std :: endl;

In file included from input_line_5:1:
In file included from /home/eliastocco/anaconda3/envs/xeus-cling/include/xeus/xinterpreter.hpp:13:
In file included from /home/eliastocco/anaconda3/envs/xeus-cling/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/functional:54:
In file included from /home/eliastocco/anaconda3/envs/xeus-cling/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/tuple:39:
In file included from /home/eliastocco/anaconda3/envs/xeus-cling/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/array:39:
In file included from /home/eliastocco/anaconda3/envs/xeus-cling/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/stdexcept:39:
In file included from /home/eliastocco/anaconda3/envs/xeus-cling/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../

In [8]:
std :: cout << typeid(matrix).name() << std :: endl ; 
std :: cout << typeid(matrix2).name() << std :: endl ; 
std :: cout << typeid(matrix3).name() << std :: endl ; 

N10eliastocco9container7dynamic14dynamic_vectorILm2EdSt6vectorEE
N10eliastocco9container7dynamic14dynamic_vectorILm2EdSt6vectorEE
N10eliastocco9container7dynamic14dynamic_vectorILm2EdSt5dequeEE


In [9]:
dynamic_vector < 4 , double > matrix4 = {{ 2 , 3 , 4 , 5 }};

In [10]:
matrix4[0]

{ { { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 } }, { { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 } }, { { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }, { 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 } } }

In [11]:
std :: array < long unsigned int , 4 > I = {1,2,3,4}

In [12]:
matrix4[I]

In file included from input_line_7:2:
header/dynamic_vector.h:61:40: error: no viable overloaded operator[] for type 'const
      __gnu_cxx::__alloc_traits<std::allocator<eliastocco::container::dynamic::dynamic_vector<1, double, vector> >
      >::value_type' (aka 'const eliastocco::container::dynamic::dynamic_vector<1, double, vector>')
                                return my_dynamic_vector 
                                       ^~~~~~~~~~~~~~~~~
header/dynamic_vector.h:61:40: note: in instantiation of member function 'eliastocco::container::dynamic::dynamic_vector<2, double, std::vector>::operator[]'
      requested here
header/dynamic_vector.h:61:40: note: in instantiation of member function 'eliastocco::container::dynamic::dynamic_vector<3, double, vector>::operator[]'
      requested here
input_line_21:2:9: note: in instantiation of member function 'eliastocco::container::dynamic::dynamic_vector<4, double, std::vector>::operator[]'
      requested here
 matrix4[I]
        ^
hea

Interpreter Error: 

In [12]:
dynamic_vector < 2 , std :: valarray<double> > Matrix = {{ 2 , 3 }};

In [15]:
Matrix[0][1].size()

0